# Gravitee.io setup

## Env Init and MongoDB setup

In [29]:
export KUBECONFIG=`pwd`/kubeconfig.yaml
export PATH=`pwd`:$PATH
echo $KUBECONFIG

/home/pierre/workspace/articles/k8s_201/kubeconfig.yaml


First we'll create a secret for our MongoDB database

In [30]:
cat > config.yml <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: gravitee-mongodb
type: Opaque
data:
  database: gravitee
stringData:
  username: gravitee
  password: "Sup3rStr0ngPassw0rd"
EOF

In [31]:
kubectl apply -f config.yml

secret/gravitee-mongodb configured


We need now to create a user account associated with the MongoDB database we'll be using

In [32]:
# Admin credential
MONGODB_ADMIN_USER=root
MONGODB_ADMIN_PASS=$(kubectl get secret my-mongodb -o jsonpath="{.data.mongodb-root-password}" | base64 --decode)
# DB connection settings
MONGODB_HOST=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")
MONGODB_PORT=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].spec.ports[0].port}")
# Gravitee credential
MONGODB_DB=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.database}")
MONGODB_GRAVITEE_USER=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.username}" | base64 --decode)
MONGODB_GRAVITEE_PASS=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.password}" | base64 --decode)
echo Host: $MONGODB_HOST, Port: $MONGODB_PORT

Host: my-mongodb.default.svc.cluster.local, Port: 27017


In [33]:
# https://docs.mongodb.com/manual/reference/built-in-roles/#database-user-roles
cat > config.js <<EOF
db = db.getSiblingDB("$MONGODB_DB")
db.dropUser("$MONGODB_GRAVITEE_USER")
db.createUser({"user": "$MONGODB_GRAVITEE_USER", "pwd": "$MONGODB_GRAVITEE_PASS", "roles": ["dbOwner"]})
EOF

In [34]:
cat config.js | kubectl run my-mongodb-client --rm -i --restart='Never' --image bitnami/mongodb --command -- mongo --host $MONGODB_HOST:$MONGODB_PORT -u $MONGODB_ADMIN_USER -p $MONGODB_ADMIN_PASS

If you don't see a command prompt, try pressing enter.
gravitee
false
Successfully added user: { "user" : "gravitee", "roles" : [ "dbOwner" ] }
bye
2019-03-14T16:53:55.634+0000 E -        [main] Error saving history file: FileOpenFailed: Unable to open() file //.dbshell: Unknown error
pod "my-mongodb-client" deleted


## ElasticSearch setup

We need to gather some ElasticSearch configuration values

In [35]:
ES_PROTOCOL=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].name}")
ES_PORT=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].port}")
ES_HOST=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")

In [36]:
echo Protocol: $ES_PROTOCOL, Host: $ES_HOST, Port: $ES_PORT

Protocol: http, Host: elasticsearch-master-headless.default.svc.cluster.local, Port: 9200


## Gravitee installation

The first step is to build the helm [chart](https://github.com/gravitee-io/gravitee-kubernetes/tree/master/gravitee).

In [37]:
curl -OL https://github.com/gravitee-io/gravitee-kubernetes/archive/master.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   136    0   136    0     0    281      0 --:--:-- --:--:-- --:--:--   282
100 26238    0 26238    0     0  26449      0 --:--:-- --:--:-- --:--:--  101k


In [38]:
unzip -u master.zip

Archive:  master.zip
1d3ae9520d30d9e9257ef13798045e9c740e3311


In [39]:
helm package gravitee-kubernetes-master/gravitee/ # build chart

Successfully packaged chart and saved it to: /home/pierre/workspace/articles/k8s_201/gravitee-1.22.0.tgz


In [40]:
helm del --purge gravitee
helm install --name gravitee gravitee-1.22.0.tgz \
--set mongo.rsEnabled=false \
--set mongo.dbhost=$MONGODB_GRAVITEE_USER:$MONGODB_GRAVITEE_PASS@$MONGODB_HOST \
--set mongo.dbport=$MONGODB_PORT \
--set es.protocol=$ES_PROTOCOL \
--set es.host=$ES_HOST \
--set es.port=$ES_PORT


Error: release: "gravitee" not found
NAME:   gravitee
LAST DEPLOYED: Thu Mar 14 17:54:12 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME              DATA  AGE
gravitee-api      1     1s
gravitee-gateway  1     1s
gravitee-ui       4     1s

==> v1/Pod(related)
NAME                               READY  STATUS             RESTARTS  AGE
gravitee-api-7ccb5794b5-sl75d      0/1    ContainerCreating  0         1s
gravitee-gateway-864d77d57d-qmxcb  0/1    ContainerCreating  0         1s
gravitee-gateway-864d77d57d-xks8b  0/1    ContainerCreating  0         1s
gravitee-ui-5cff4d56f5-dzffp       0/1    ContainerCreating  0         1s

==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)   AGE
gravitee-api      ClusterIP  10.3.60.160   <none>       83/TCP    1s
gravitee-gateway  ClusterIP  10.3.150.156  <none>       82/TCP    1s
gravitee-ui       ClusterIP  10.3.230.157  <none>       8002/TCP  1s

==> v1beta1/Deployment
NAME              READY

Check that the pods are up and running

In [41]:
kubectl get pods -l release=gravitee

NAME                                READY   STATUS    RESTARTS   AGE
gravitee-api-7ccb5794b5-sl75d       0/1     Running   0          6s
gravitee-gateway-864d77d57d-qmxcb   0/1     Running   0          6s
gravitee-gateway-864d77d57d-xks8b   0/1     Running   0          6s
gravitee-ui-5cff4d56f5-dzffp        0/1     Running   0          6s
